# 01. LangGraph Basics - Khái niệm cơ bản

## Mục tiêu học tập
- Hiểu được LangGraph là gì và tại sao nó quan trọng
- Nắm vững các khái niệm cốt lõi: Graph, Nodes, Edges, State
- Tạo được một Graph cơ bản đầu tiên
- Hiểu luồng điều khiển trong LangGraph

## Giới thiệu về LangGraph

### LangGraph là gì?
LangGraph là một thư viện để xây dựng các ứng dụng có trạng thái (stateful) và đa tác nhân (multi-actor) với các LLM. Nó mở rộng LangChain Expression Language với khả năng:
- Định nghĩa luồng xử lý phức tạp dưới dạng Graph
- Quản lý trạng thái (state) xuyên suốt quá trình xử lý
- Hỗ trợ các tác nhân (agents) có thể tương tác với nhau

### Tại sao cần LangGraph?
- **Kiểm soát luồng xử lý**: Thay vì chuỗi tuyến tính, bạn có thể tạo luồng phức tạp với điều kiện và vòng lặp
- **Quản lý trạng thái**: Duy trì thông tin xuyên suốt quá trình xử lý
- **Khả năng mở rộng**: Dễ dàng thêm các bước xử lý mới
- **Debug và theo dõi**: Có thể quan sát từng bước trong quá trình xử lý

## Các khái niệm cốt lõi

### 1. Graph (Đồ thị)
- Cấu trúc chính của LangGraph
- Bao gồm các nodes (nút) và edges (cạnh)
- Định nghĩa luồng xử lý từ đầu đến cuối

### 2. Nodes (Nút)
- Đại diện cho các bước xử lý
- Có thể là function Python, LLM call, hay bất kỳ logic nào
- Nhận state làm input và trả về state mới

### 3. Edges (Cạnh)
- Kết nối các nodes với nhau
- Định nghĩa thứ tự thực hiện
- Có thể có điều kiện (conditional edges)

### 4. State (Trạng thái)
- Dữ liệu được truyền qua các nodes
- Có thể được cập nhật tại mỗi node
- Thường là dictionary hoặc TypedDict

## Cài đặt & Cấu hình

In [ ]:
# Cài đặt các thư viện cần thiết
!pip install langgraph langchain langchain-anthropic python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Kiểm tra API key
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
if not anthropic_api_key:
    print("⚠️ Cần cấu hình ANTHROPIC_API_KEY trong file .env")
    print("Tạo file .env và thêm: ANTHROPIC_API_KEY=your_api_key_here")
else:
    print("✅ ANTHROPIC_API_KEY đã được cấu hình")

## Import các thư viện cần thiết

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, AIMessage
import operator

## Ví dụ 1: Định nghĩa State

In [ ]:
# Định nghĩa State schema
class GraphState(TypedDict):
    # Danh sách tin nhắn
    messages: Annotated[list, add_messages]
    # Bước xử lý hiện tại
    current_step: str
    # Số lần xử lý
    step_count: int

print("✅ Đã định nghĩa GraphState")

## Ví dụ 2: Định nghĩa các Node (Functions)

In [ ]:
# Node 1: Khởi tạo
def initialize_node(state: GraphState) -> GraphState:
    """
    Node khởi tạo - bước đầu tiên trong graph
    """
    print("🚀 Bắt đầu xử lý...")
    
    return {
        "current_step": "initialized",
        "step_count": 1,
        "messages": [HumanMessage(content="Hệ thống đã khởi tạo")]
    }

# Node 2: Xử lý với LLM
def llm_node(state: GraphState) -> GraphState:
    """
    Node xử lý với LLM
    """
    print(f"🤖 Xử lý với LLM (bước {state['step_count'] + 1})...")
    
    # Khởi tạo LLM
    llm = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0)
    
    # Tạo prompt
    prompt = "Bạn là một trợ lý AI thân thiện. Hãy chào hỏi người dùng bằng tiếng Việt."
    
    # Gọi LLM
    response = llm.invoke([HumanMessage(content=prompt)])
    
    return {
        "current_step": "llm_processed",
        "step_count": state["step_count"] + 1,
        "messages": [response]
    }

# Node 3: Hoàn thành
def finalize_node(state: GraphState) -> GraphState:
    """
    Node hoàn thành - bước cuối cùng
    """
    print(f"✅ Hoàn thành xử lý (tổng cộng {state['step_count'] + 1} bước)")
    
    return {
        "current_step": "completed",
        "step_count": state["step_count"] + 1,
        "messages": state["messages"] + [HumanMessage(content="Xử lý hoàn tất")]
    }

print("✅ Đã định nghĩa 3 nodes: initialize_node, llm_node, finalize_node")

## Ví dụ 3: Tạo Graph cơ bản

In [ ]:
# Tạo StateGraph
workflow = StateGraph(GraphState)

# Thêm các nodes
workflow.add_node("initialize", initialize_node)
workflow.add_node("llm_process", llm_node)
workflow.add_node("finalize", finalize_node)

# Định nghĩa luồng xử lý (edges)
workflow.add_edge(START, "initialize")  # Bắt đầu -> initialize
workflow.add_edge("initialize", "llm_process")  # initialize -> llm_process
workflow.add_edge("llm_process", "finalize")  # llm_process -> finalize
workflow.add_edge("finalize", END)  # finalize -> kết thúc

# Compile graph
app = workflow.compile()

print("✅ Đã tạo thành công LangGraph với 3 nodes")
print("📊 Luồng xử lý: START → initialize → llm_process → finalize → END")

## Ví dụ 4: Chạy Graph

In [ ]:
# State ban đầu
initial_state = {
    "messages": [],
    "current_step": "start",
    "step_count": 0
}

print("🎯 Chạy LangGraph...")
print("=" * 50)

# Chạy graph
final_state = app.invoke(initial_state)

print("=" * 50)
print("📋 Kết quả cuối cùng:")
print(f"   - Bước hiện tại: {final_state['current_step']}")
print(f"   - Số bước đã xử lý: {final_state['step_count']}")
print(f"   - Số tin nhắn: {len(final_state['messages'])}")

# Hiển thị tin nhắn cuối cùng từ LLM
if final_state['messages']:
    last_ai_message = None
    for msg in final_state['messages']:
        if isinstance(msg, AIMessage):
            last_ai_message = msg
    
    if last_ai_message:
        print(f"\n🤖 Phản hồi từ AI: {last_ai_message.content}")

## Giải thích & Phân tích

### Cách hoạt động của Graph

1. **Khởi tạo State**: Graph bắt đầu với state rỗng
2. **Chạy từng Node**: Mỗi node nhận state hiện tại, xử lý và trả về state mới
3. **Cập nhật State**: State được cập nhật sau mỗi node
4. **Chuyển đổi Node**: Edges định nghĩa node tiếp theo sẽ được thực hiện
5. **Kết thúc**: Graph kết thúc khi đến END node

### Ưu điểm của cách tiếp cận này

- **Rõ ràng**: Mỗi bước xử lý được định nghĩa riêng biệt
- **Có thể theo dõi**: Bạn có thể thấy state thay đổi như thế nào
- **Linh hoạt**: Dễ dàng thêm/bớt nodes hoặc thay đổi luồng
- **Tái sử dụng**: Các nodes có thể được sử dụng lại trong các graphs khác

## Ví dụ 5: Xem cấu trúc Graph

In [ ]:
# In ra cấu trúc của graph
print("📊 Cấu trúc Graph:")
print(f"   - Nodes: {list(app.graph.nodes.keys())}")
print(f"   - Edges: {len(app.graph.edges)} kết nối")

# Xem chi tiết các edges
print("\n🔗 Chi tiết các kết nối:")
for edge in app.graph.edges:
    print(f"   {edge[0]} → {edge[1]}")

## Ví dụ 6: Chạy từng bước (Streaming)

In [ ]:
print("🔄 Chạy Graph với streaming (xem từng bước):")
print("=" * 60)

# State ban đầu
streaming_state = {
    "messages": [],
    "current_step": "start",
    "step_count": 0
}

# Chạy với streaming để xem từng bước
for i, step in enumerate(app.stream(streaming_state)):
    print(f"\n📍 Bước {i+1}:")
    for node_name, node_output in step.items():
        print(f"   Node: {node_name}")
        print(f"   Current step: {node_output.get('current_step', 'N/A')}")
        print(f"   Step count: {node_output.get('step_count', 'N/A')}")
        print(f"   Messages: {len(node_output.get('messages', []))}")

print("\n✅ Hoàn thành streaming")

## Kết luận & Bước tiếp theo

### Những gì đã học được

1. **Khái niệm cơ bản**: Graph, Nodes, Edges, State
2. **Cách tạo Graph**: Sử dụng StateGraph và compile
3. **Định nghĩa Nodes**: Các function Python nhận và trả về state
4. **Kết nối Nodes**: Sử dụng add_edge để định nghĩa luồng
5. **Chạy Graph**: invoke() và stream() để thực hiện

### Bước tiếp theo

Trong các notebook tiếp theo, chúng ta sẽ tìm hiểu:
- **Conditional Edges**: Tạo luồng có điều kiện
- **Human-in-the-loop**: Tương tác với người dùng
- **Multi-agent systems**: Nhiều agent hoạt động cùng lúc
- **Persistence**: Lưu trữ và khôi phục state
- **Error handling**: Xử lý lỗi trong graph

### Bài tập thực hành

1. Thêm một node mới vào graph hiện tại
2. Tạo một graph với 2 nhánh parallel
3. Thử thay đổi thứ tự các nodes
4. Tạo một graph xử lý danh sách items

---

**📚 Tài liệu tham khảo:**
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [LangGraph GitHub](https://github.com/langchain-ai/langgraph)
- [LangChain Expression Language](https://python.langchain.com/docs/expression_language/)

**💡 Tips:**
- Luôn định nghĩa State schema rõ ràng
- Tên nodes nên mô tả rõ chức năng
- Sử dụng streaming để debug
- Test từng node riêng biệt trước khi kết hợp